<a href="https://colab.research.google.com/github/kaanbaycan/IE-423-Forecasting-Tasks/blob/main/IE_423_Task_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trump Tweets Dataset Import

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
def upload_dataset(from_drive = False):
  if not os.path.exists("kaggle.json"):
    if not from_drive:
      print("Upload Kaggle API Key")
      files.upload()
      print("Downloading dataset...")
    else:
      !cp /content/drive/MyDrive/kaggle.json /content/
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d austinreese/trump-tweets
    !unzip trump-tweets.zip
  else:
    print("Dataset already exists")

In [ ]:
upload_dataset()

Upload Kaggle API Key


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/austinreese/trump-tweets
License(s): CC0-1.0
100% 6.88M/6.88M [00:00<00:00, 34.6MB/s]
100% 6.88M/6.88M [00:00<00:00, 30.5MB/s]
Archive:  trump-tweets.zip
  inflating: realdonaldtrump.csv     
  inflating: trumptweets.csv         


# EDA

In [ ]:
df = pd.read_csv("realdonaldtrump.csv")
#lets only look at the inital 100 rows
df = df.head(100)

In [ ]:
df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


We have the content, date, retweet, fav, mention and hastaghs nos in the dataset. Today, I will only play with the content.

In [ ]:
df.drop(["id","link","retweets","favorites","mentions","hashtags"],axis=1,inplace=True)

In [ ]:
df.head()

,content,date
0,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25
1,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10
2,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08
3,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15
4,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28


# Sentiment Analysis w Donald Trump's Tweets

In [ ]:
from transformers import pipeline

In [ ]:
sentiment = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
df["Sentiment"] = df["content"].apply(lambda x: sentiment(x)[0]["label"])

We can look at the corralation between nasdaq or S&P indexes and Donald Trumps tweets sentiment.

In [ ]:
print("Some positive labelled tweets")
print(df[df["Sentiment"] == "POSITIVE"].sample(10)["content"].values)
print("Some negative labelled tweets")
print(df[df["Sentiment"] == "NEGATIVE"].sample(10)["content"].values)

Some positive labelled tweets
["Trump Tycoon App for iPhone & iPod Touch - It's $2.99 but the advice is priceless! http://bit.ly/UGUF0"
 'Congrats to winners from around the world who entered the "Think Like A Champion" signed book/keychain contest! http://tinyurl.com/pqpfvm'
 "Tune in to The Marriage Ref onThursday night at 10 p.m. on NBC--I'm on the panel of experts along with Gloria Estefan & Adam Carolla."
 'Miss USA Tara Conner will not be fired - "I\'ve always been a believer in second chances." says Donald Trump'
 'From Donald Trump: Andrea Bocelli @Mar-a-Lago - Many say best night of entertainment in long history of Palm Beach http://bit.ly/2fNgOz'
 'Read an excerpt from Think Like A Champion by Donald J. Trump: http://bit.ly/11FHpR'
 'Wishing you and yours a very Happy and Bountiful Thanksgiving!'
 "Don't forget the open call at Trump Tower tomorrow for The Apprentice. I look forward to seeing you there."
 'Be sure to look for my beautiful wife Melania Trump tonight on QVC at 

# Sentence Similarity of Tweets

In [ ]:
pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
#Lets look at the similarities between sentences
df.head(20)

,content,date,Sentiment
0,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,POSITIVE
1,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,POSITIVE
2,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,POSITIVE
3,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,NEGATIVE
4,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,NEGATIVE
5,"Miss USA Tara Conner will not be fired - ""I've...",2009-05-12 14:21:55,POSITIVE
6,Listen to an interview with Donald Trump discu...,2009-05-13 12:38:28,POSITIVE
7,"""Strive for wholeness and keep your sense of w...",2009-05-14 11:30:40,POSITIVE
8,"Enter the ""Think Like A Champion"" signed book ...",2009-05-15 09:13:13,POSITIVE
9,"""When the achiever achieves, it's not a platea...",2009-05-16 17:22:45,NEGATIVE


In [ ]:
embed_1 = model.encode(df.iloc[0]["content"])
embed_2 = model.encode(df.iloc[1]["content"])
print(embed_1 @ embed_2.T)

0.64938223


In [ ]:
embed_3 = model.encode(df.iloc[2]["content"])
embed_4 = model.encode(df.iloc[3]["content"])
print(embed_3 @ embed_4.T)

0.5337726


In [ ]:
embed_5 = model.encode(df.iloc[14]["content"])
embed_6 = model.encode(df.iloc[19]["content"])
print(embed_5 @ embed_6.T)

0.6683404


In [ ]:
embed_7 = model.encode(df.iloc[18]["content"])
embed_8 = model.encode(df.iloc[6]["content"])
print(embed_7 @ embed_8.T)

0.627043


# Summarization of a part from Tüpraş Investor Presentations

Now I will upload a texture from Tupras Investor Relations and ask the model to summarize. This can be beneficial for large amount of statement summaries.

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
text = """
Goal
We are changing and transforming for a sustainable
world under the banner of “Energy to the Future”.
We are pledging to become the Leading Carbon
Neutral Energy Company by 2050, with the strength
we derive from our deeply rooted experience going
Being People-Centric:
We have designed the Tüpraş of the future
with an approach that is people-centric, acts
with the awareness of responsible production,
and prioritizes inclusion in social benefit and
the creation of value for its stakeholders.
Recognizing that our colleagues represent our
most valuable asset, we keep the “My Future
is My Safety” approach at the center of all
our processes in a business environment that
is egalitarian, inclusive, fair and respectful of
human rights. Deriving strength from our R&D,
innovation and digitalization culture, we support
the development of our human resources who
possess the knowledge and skills compatible
with the business models of the future.
identity as Turkey’s largest industrial enterprise,
and the responsibilities which we have taken on
for contributing to the circular economy and for
our low-carbon transition journey. We aim to lead
the clean energy transformation. In line with our
Strategic Transition Plan, we are focusing on four
Transformation for the Energy of the Future:
We aim to achieve sustainable and profitable
growth with our balanced business portfolio by
creating the highest added value for our country,
the shareholders, employees and business
partners. We continue to work with all our might
with the aim of transforming into a carbon-neutral
energy company by developing our transition
strategy efficiently, flexibly and sustainably.
Values
Safety
• Safety is our priority.
• Safety is our responsibility.
• We know the ten mandatory rules and
implement them.
• We act rapidly to correct unsafe situations.
Honesty
• We are honest with each other.
• We know our responsibility and learn from
our mistakes.
• We trust each other.
• We protect Tüpraş and each other.
GENERAL ASSEMBLY
ABOUT TÜPRAŞ
core business lines that we identified as “Sustainable
Refining”, “Biofuels”, “Zero-Carbon Electricity”
and “Green Hydrogen” based on the fundamental
approach of “having a well-balanced and diversified
clean energy portfolio that supports our sustainability
understanding”. While meeting our country’s energy
needs without interruption, we work with all our might
Responsible Production: We act with the
awareness that sustainable growth is only
possible by protecting the environment and
natural resources. We closely monitor national
and international current agendas and policies
regarding climate change. We use resources
efficiently in all our processes and contribute to
the circular economy. We work to improve the
environmental and social impacts of our value
chain.
Respect
• We comply with the Tüpraş policy on
Diversity, Equality and Inclusion (D/E/I).
• We communicate respectfully with each
other.
• We take care of each other.
to generate sustainable value with our employees,
business partners, suppliers and stakeholders.
We have designed the Tüpraş of the future with
an approach that is people-centric, acts with the
awareness of responsible production and prioritizes
inclusion in social benefit and the creation of value for
its stakeholders.
Inclusion in Social Benefit: We aim for inclusion
in social benefit with an approach focused on
sustainable development. We espoused the
United Nations Sustainable Development Goals
as our guide. With the awareness that comes
with being a global corporate citizen, we carry out
projects that support social development in the
fields of education, environment, equality, culture
and the arts by adopting a system of cooperation
among the business world, public sector and
NGOs.
"""


In [ ]:
pipe(text)

[{'summary_text': 'Tüpraş is Turkey’s largest industrial enterprise. The company aims to become the Leading Carbon  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Neutral Energy Company by 2050. The “My Future  is My Safety” approach is at the center of all processes. TüpRAŞ has four core business lines that we identified as “Sustainable”, “Biofuels” and “Zero-Carbon Electricity”'}]

It is able to catch some key points although it misses some of the initial details. But satisfying enough, the summary makes sense and easy to understand.

# Q/A From a Financial Statement of a Firm

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="deepset/roberta-base-squad2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
text = """
INTERIM CONSOLIDATED STATEMENTS OF PROFIT OR LOSS AND OTHER COMPREHENSIVE INCOME
(Amounts expressed in thousands of Turkish Lira (“TL”) in terms of the purchasing power of the TL at 30 June 2024, unless otherwise indicated.)
Reviewed Reviewed
1 January- 1 April- 1 January- 1 April
Notes 30 June 2024 30 June 2024 30 June 2023 30 June 2023
Revenue 21 370,096,723 190,715,672 357,349,229 183,629,533
Cost of sales 21 (338,382,791) (173,283,800) (302,275,494) (154,319,423)
Gross profit (loss) 31,713,932 17,431,872 55,073,735 29,310,110
General administrative expenses 22 (7,716,893) (3,912,415) (7,365,662) (3,261,673)
Marketing expenses 22 (5,301,148) (2,868,593) (3,384,130) (1,900,528)
Research and development expenses 22 (188,274) (80,296) (207,703) (103,015)
Other operating income 23 7,456,725 4,328,119 7,044,577 5,847,324
Other operating expenses 23 (11,071,177) (4,319,240) (19,381,694) (17,196,444)
Operating profit (loss) 14,893,165 10,579,447 31,779,123 12,695,774
Income/ (expenses) from investment activities 24 (3,525) (3,141) (2,512) (19,485)
Income (loss) from investments accounted by equity method 10 306,225 567 927,242 1,391,954
Operating profit before financial income (expense) 15,195,865 10,576,873 32,703,853 14,068,243
Financial income 25 14,294,201 6,107,607 13,883,863 10,736,299
Financial expense 25 (9,679,935) (5,006,243) (19,114,521) (14,527,109)
Monetary loss/gain (8,767,673) (2,686,906) (4,616,623) (945,184)
Profit (loss) before tax from continued operations 11,042,458 8,991,331 22,856,572 9,332,249
Tax income (expense) (5,521,011) (3,920,028) (10,220,000) 186,431
Taxes on income (expense) (2,023,736) (1,454,133) (6,269,322) (624,104)
Deferred tax income (expense) 26 (3,497,275) (2,465,895) (3,950,678) 810,535
Net income (loss) from continued operations 5,521,447 5,071,303 12,636,572 9,518,680"""

In [ ]:
pipe.__call__(question="What is the total revenue?", context=text)

{'score': 0.35086172819137573,
 'start': 352,
 'end': 363,
 'answer': '370,096,723'}

In [ ]:
pipe.__call__(question="Is net income increasesd?", context=text)

{'score': 0.01705806329846382,
 'start': 1754,
 'end': 1797,
 'answer': 'Net income (loss) from continued operations'}

This models fails to analyze data, only reads and able to give information as it can answer what is total revenue question but not that if it is increased or not. Lets try another model.

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
pipe.__call__(question="What is the total revenue?", context=text)

{'score': 0.10732144862413406,
 'start': 349,
 'end': 363,
 'answer': '21 370,096,723'}

In [ ]:
pipe.__call__(question="What are the last 4 net incomes", context=text)

{'score': 0.0005171020166017115,
 'start': 703,
 'end': 723,
 'answer': 'development expenses'}

Again the model lacks to analyze the data. Hence we understand that these models do not work like ChatGPT, rather takes the text and tries to understand key points. Then only can give out these key points. Hence the importance of GPT if felt here.